In [ ]:
!pip install pardata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install welly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pardata
import os
import requests
import tarfile
import pandas as pd
import numpy as np
import welly.quality as wq
from IPython.display import HTML, display
import welly
import matplotlib.pyplot as plt

In [ ]:
# Download the dataset
fname = 'taranaki-basin-curated-well-logs.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-taranaki-basin-curated-well-logs/1.0.0/'
download_link = url + fname
r = requests.get(download_link, allow_redirects=True)
open(fname , 'wb').write(r.content)

228743380

In [ ]:
tar = tarfile.open(fname)
tar.extractall()
tar.close()

In [ ]:
data_path = "taranaki-basin-curated-well-logs"
os.path.exists(data_path)

True

In [ ]:
logs_df = pd.read_csv(data_path + '/logs.csv')

<ipython-input-13-1630b448bbec>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  logs_df = pd.read_csv(data_path + '/logs.csv')


In [ ]:
logs_df.set_index(['WELLNAME', 'DEPT'], inplace=True)

In [ ]:
logs_df.head()

BS  CALI  DENS  DRHO  DTC  GR  NEUT  PEF  RESD  \
WELLNAME         DEPT                                                         
Ahuroa South B-1 27.5844  12.25   NaN   NaN   NaN  NaN NaN   NaN  NaN   NaN   
                 27.7368  12.25   NaN   NaN   NaN  NaN NaN   NaN  NaN   NaN   
                 27.8892  12.25   NaN   NaN   NaN  NaN NaN   NaN  NaN   NaN   
                 28.0416  12.25   NaN   NaN   NaN  NaN NaN   NaN  NaN   NaN   
                 28.1940  12.25   NaN   NaN   NaN  NaN NaN   NaN  NaN   NaN   

                          RESM  RESS  SP     TEMP  TENS             X  \
WELLNAME         DEPT                                                   
Ahuroa South B-1 27.5844   NaN   NaN NaN      NaN   NaN  424403.93838   
                 27.7368   NaN   NaN NaN      NaN   NaN  424403.93838   
                 27.8892   NaN   NaN NaN      NaN   NaN  424403.93838   
                 28.0416   NaN   NaN NaN  33.0429   NaN  424403.93838   
                 28.1940   NaN   NaN NaN  33.0429   NaN  424403.93838   

                                    Y         Z  ONSHORE  DIRSURVEY FORMATION  
WELLNAME         DEPT                                                          
Ahuroa South B-1 27.5844  399869.5574  229.4244     True      False       NaN  
                 27.7368  399869.5574  229.5768     True      False       NaN  
                 27.8892  399869.5574  229.7292     True      False       NaN  
                 28.0416  399869.5574  229.8816     True      False       NaN  
                 28.1940  399869.5574  230.0340     True      False       NaN

In [ ]:
# Расчет частот каротажей в скважинах без поностью пропущенных значений
count_logs = {}
for well in logs_df.index.get_level_values(0).unique():
  for colum in logs_df.columns:
    if logs_df.loc[well][colum].notnull().any() == True:
      if colum in count_logs:
            count_logs[colum] += 1
      else:
            count_logs[colum] = 1
result_dict = {key: value / 407 for key, value in count_logs.items()}
dict(sorted(result_dict.items(), key = lambda x: x[1], reverse = True))

{'BS': 1.0,
 'X': 1.0,
 'Y': 1.0,
 'Z': 1.0,
 'ONSHORE': 1.0,
 'DIRSURVEY': 1.0,
 'GR': 0.9975429975429976,
 'RESD': 0.9262899262899262,
 'FORMATION': 0.8968058968058968,
 'CALI': 0.8697788697788698,
 'DENS': 0.8378378378378378,
 'RESS': 0.8378378378378378,
 'DTC': 0.8329238329238329,
 'NEUT': 0.8304668304668305,
 'DRHO': 0.7862407862407862,
 'SP': 0.7641277641277642,
 'TENS': 0.7542997542997543,
 'PEF': 0.6339066339066339,
 'TEMP': 0.6314496314496314,
 'RESM': 0.5085995085995086}

In [ ]:
df = logs_df[['CALI', 'DENS', 'DRHO', 'GR', 'NEUT', 'SP']]

In [ ]:
df.head()

CALI  DENS  DRHO  GR  NEUT  SP
WELLNAME         DEPT                                   
Ahuroa South B-1 27.5844   NaN   NaN   NaN NaN   NaN NaN
                 27.7368   NaN   NaN   NaN NaN   NaN NaN
                 27.8892   NaN   NaN   NaN NaN   NaN NaN
                 28.0416   NaN   NaN   NaN NaN   NaN NaN
                 28.1940   NaN   NaN   NaN NaN   NaN NaN

In [ ]:
for well in df.index.get_level_values(0).unique():
    for colum in df.columns:
      if df.loc[well][colum].notnull().any() != True:
        df = df.drop(index = well, level = 'WELLNAME')
        break

In [ ]:
df_test = df.copy()

In [ ]:
# Отбор скважин, в которых каротажи не имеют полностью пропущенных значений
def values_nan(df, dept_1, dept_2, well):
    depth_mask = (df.index.get_level_values(1) < dept_1) | (df.index.get_level_values(1) > dept_2)
    mask = (df.index.get_level_values(0) == well) & depth_mask
    df = df[~mask | (df.index.get_level_values(0) != well)]
    return df

In [ ]:
nan_list = [{'dept_1': 659.5872, 'dept_2': 2369.9724, 'well': 'Ahuroa South B-1'},
            {'dept_1': 2519.0196, 'dept_2': 3157.8804, 'well': 'Ahuroa-1'},
            {'dept_1': 2830.0680, 'dept_2': 3112.4652, 'well': 'Ahuroa-1A'},
            {'dept_1': 1810.5120, 'dept_2': 2434.4376, 'well': 'Ahuroa-2'},
            {'dept_1': 2272.5888, 'dept_2': 2523.8964, 'well': 'Ahuroa-4'},
            {'dept_1': 972.6168, 'dept_2': 2506.3704, 'well': 'Ahuroa-5'},
            {'dept_1': 973.0740, 'dept_2': 2379.2688, 'well': 'Ahuroa-5 ST1'},
            {'dept_1': 1173.6324, 'dept_2': 2128.4184, 'well': 'Albacore-1'},
            {'dept_1': 1501.1400, 'dept_2': 3081.3756, 'well': 'Amokura-1'},
            {'dept_1': 604.5708, 'dept_2': 2360.9808, 'well': 'Arakamu-1'},
            {'dept_1': 1501.5972, 'dept_2': 3021.4824, 'well': 'Arawa-1'},
            {'dept_1': 482.0412, 'dept_2': 3497.8848, 'well': 'Ariki-1'},
            {'dept_1': 1570.0248, 'dept_2': 3240.4812, 'well': 'Awatea-1'},
            {'dept_1': 395.4780, 'dept_2': 1431.9504, 'well': 'Bluff-1'},
            {'dept_1': 452.6280, 'dept_2': 1485.9000, 'well': 'Burgess-1'},
            {'dept_1': 1689.0492, 'dept_2': 2407.9200, 'well': 'Cape Egmont-1'},
            {'dept_1': 1581.6072, 'dept_2': 2808.4272, 'well': 'Cape Farewell-1'},
            {'dept_1': 1702.6128, 'dept_2': 3370.6308, 'well': 'Cardiff-1'},
            {'dept_1': 387.5532, 'dept_2': 1842.3636, 'well': 'Cheal B-2'},
            {'dept_1': 728.6244, 'dept_2': 1901.9520, 'well': 'Cheal B-3'},
            {'dept_1': 1505.1024, 'dept_2': 2833.8780, 'well': 'Cheal-1'},
            {'dept_1': 1296.0096, 'dept_2': 1929.9936, 'well': 'Cheal-2'},
            {'dept_1': 404.6220, 'dept_2': 1865.9856, 'well': 'Cheal-A3X'},
            {'dept_1': 404.6220, 'dept_2': 1825.4472, 'well': 'Cheal-A4'},
            {'dept_1': 1284.5796, 'dept_2': 1875.5868, 'well': 'Cheal-A6'},
            {'dept_1': 1252.5756, 'dept_2': 1907.4384, 'well': 'Cheal-A6 ST1'},
            {'dept_1': 1248.6132, 'dept_2': 1821.4848, 'well': 'Cheal-A7'},
            {'dept_1': 268.0716, 'dept_2': 1532.3820, 'well': 'Cheal-B-H1-MM'},
            {'dept_1': 423.5196, 'dept_2': 1784.9088, 'well': 'Cheal-B1'},
            {'dept_1': 791.1084, 'dept_2': 2254.4532, 'well': 'Cheal-B4'},
            {'dept_1': 460.5528, 'dept_2': 1770.8880, 'well': 'Clematis-1'},
            {'dept_1': 570.5856, 'dept_2': 2420.4168, 'well': 'Crusader-1'},
            {'dept_1': 1102.6140, 'dept_2': 2276.8560, 'well': 'Durham-1'},
            {'dept_1': 402.6408, 'dept_2': 980.9988, 'well': 'Fresne-1'},
            {'dept_1': 439.5216, 'dept_2': 2001.9264, 'well': 'Goldie-1'},
            {'dept_1': 819.1500, 'dept_2': 2733.4464, 'well': 'Goss-A1'},
            {'dept_1': 3581.5524, 'dept_2': 3857.8536, 'well': 'Goss-A1 ST1'},
            {'dept_1': 406.6032, 'dept_2': 1411.9860, 'well': 'Heaphy-1'},
            {'dept_1': 451.5612, 'dept_2': 1906.9812, 'well': 'Hihi-1'},
            {'dept_1': 451.5612, 'dept_2': 1769.3640, 'well': 'Hihi-1A'},
            {'dept_1': 1400.5560, 'dept_2': 2767.4316, 'well': 'Hochstetter-1'},
            {'dept_1': 1506.3216, 'dept_2': 2592.9336, 'well': 'Hoki-1'},
            {'dept_1': 632.6124, 'dept_2': 1651.4064, 'well': 'Honeysuckle-1'},
            {'dept_1': 431.1396, 'dept_2': 1793.9004, 'well': 'Hursthouse-1'},
            {'dept_1': 1609.0392, 'dept_2': 3408.8832, 'well': 'Kahawai-1'},
            {'dept_1': 443.6364, 'dept_2': 2395.4232, 'well': 'Kahili-1'},
            {'dept_1': 865.6320, 'dept_2': 2424.3792, 'well': 'Kahili-1A'},
            {'dept_1': 2486.5584, 'dept_2': 3009.4428, 'well': 'Kahili-1B'},
            {'dept_1': 1873.1484, 'dept_2': 2450.4396, 'well': 'Kahili-1C'},
            {'dept_1': 2923.0320, 'dept_2': 3812.4384, 'well': 'Kahu-1'},
            {'dept_1': 607.6188, 'dept_2': 1542.4404, 'well': 'Kaimiro-10'},
            {'dept_1': 611.5812, 'dept_2': 1788.8712, 'well': 'Kaimiro-12'},
            {'dept_1': 614.0196, 'dept_2': 1596.9996, 'well': 'Kaimiro-13'},
            {'dept_1': 604.5708, 'dept_2': 2289.3528, 'well': 'Kaimiro-15'},
            {'dept_1': 585.5208, 'dept_2': 1825.9044, 'well': 'Kaimiro-4'},
            {'dept_1': 600.6084, 'dept_2': 1738.8840, 'well': 'Kaimiro-5'},
            {'dept_1': 614.0196, 'dept_2': 1875.8916, 'well': 'Kaimiro-6'},
            {'dept_1': 600.6084, 'dept_2': 2161.9464, 'well': 'Kaimiro-7'},
            {'dept_1': 481.5840, 'dept_2': 1992.4776, 'well': 'Kaimiro-7ST1'},
            {'dept_1': 603.5040, 'dept_2': 1725.4728, 'well': 'Kaimiro-8'},
            {'dept_1': 320.6496, 'dept_2': 1830.4764, 'well': 'Kaipikari-1'},
            {'dept_1': 606.0948, 'dept_2': 1791.4620, 'well': 'Kakariki-1'},
            {'dept_1': 555.6504, 'dept_2': 1570.4820, 'well': 'Kanuka-1'},
            {'dept_1': 494.5380, 'dept_2': 2014.4232, 'well': 'Karaka-A1'},
            {'dept_1': 753.6180, 'dept_2': 2180.3868, 'well': 'Karewa-1'},
            {'dept_1': 1606.6008, 'dept_2': 3123.8952, 'well': 'Kea-1'},
            {'dept_1': 377.6472, 'dept_2': 2066.8488, 'well': 'Konini-1'},
            {'dept_1': 1747.5708, 'dept_2': 2947.4160, 'well': 'Kora-1'},
            {'dept_1': 895.0452, 'dept_2': 2694.8892, 'well': 'Korito-1'},
            {'dept_1': 611.1240, 'dept_2': 3355.3908, 'well': 'Kowhai-A1'},
            {'dept_1': 2846.0700, 'dept_2': 3772.3572, 'well': 'Kupe South-4'},
            {'dept_1': 2262.5304, 'dept_2': 3756.9648, 'well': 'MB-Q(10)'},
            {'dept_1': 2671.1148, 'dept_2': 3373.3740, 'well': 'MB-R(1)'},
            {'dept_1': 2233.5744, 'dept_2': 4366.4124, 'well': 'MB-S(12)'},
            {'dept_1': 3437.5344, 'dept_2': 4035.3996, 'well': 'MB-W(2)'},
            {'dept_1': 695.0964, 'dept_2': 1461.3636, 'well': 'Maari-2'},
            {'dept_1': 464.5152, 'dept_2': 1121.3592, 'well': 'Maketawa-1'},
            {'dept_1': 511.6068, 'dept_2': 1577.9496, 'well': 'Mako-1'},
            {'dept_1': 490.5756, 'dept_2': 3216.4020, 'well': 'Maui-6'},
            {'dept_1': 491.0328, 'dept_2': 2113.4832, 'well': 'Maui-7'},
            {'dept_1': 1629.0036, 'dept_2': 2286.9144, 'well': 'McKee-2C'},
            {'dept_1': 29.5656, 'dept_2': 570.8904, 'well': 'McKee-6'},
            {'dept_1': 614.0196, 'dept_2': 1849.3740, 'well': 'Miromiro-1'},
            {'dept_1': 2651.1504, 'dept_2': 3784.3968, 'well': 'Mokau-1'},
            {'dept_1': 197.5104, 'dept_2': 1835.3532, 'well': 'Moturoa-6 ST1'},
            {'dept_1': 366.5220, 'dept_2': 1166.9268, 'well': 'Moturoa-7'},
            {'dept_1': 600.6084, 'dept_2': 2530.4496, 'well': 'Mystone-1'},
            {'dept_1': 584.6064, 'dept_2': 1378.4580, 'well': 'Ngaere-3'},
            {'dept_1': 418.0332, 'dept_2': 1989.4296, 'well': 'Ngatoro-13'},
            {'dept_1': 998.5248, 'dept_2': 1533.4488, 'well': 'Ngatoro-2'},
            {'dept_1': 552.6024, 'dept_2': 2394.9660, 'well': 'Ngatoro-3'},
            {'dept_1': 646.0236, 'dept_2': 1890.9792, 'well': 'Ngatoro-3ST1'},
            {'dept_1': 1558.1376, 'dept_2': 2722.9308, 'well': 'North Tasman-1'},
            {'dept_1': 1402.5372, 'dept_2': 2351.3796, 'well': 'Oakura-1'},
            {'dept_1': 1418.5392, 'dept_2': 3717.4932, 'well': 'Ohanga-2'},
            {'dept_1': 1808.0736, 'dept_2': 4235.9580, 'well': 'Okoki-1'},
            {'dept_1': 451.1040, 'dept_2': 1577.9496, 'well': 'Oru-1'},
            {'dept_1': 498.5004, 'dept_2': 1882.9020, 'well': 'Paritutu-1'},
            {'dept_1': 675.5892, 'dept_2': 2507.8944, 'well': 'Piakau North-A1'},
            {'dept_1': 661.1112, 'dept_2': 2507.5896, 'well': 'Piakau North-A2'},
            {'dept_1': 404.4696, 'dept_2': 2167.4328, 'well': 'Pohokura South-1'},
            {'dept_1': 885.1392, 'dept_2': 3114.4464, 'well': 'Pohokura-3'},
            {'dept_1': 1637.5380, 'dept_2': 2218.4868, 'well': 'Pouri-2'},
            {'dept_1': 3187.1412, 'dept_2': 4175.9124, 'well': 'Pukeko-1'},
            {'dept_1': 1342.6440, 'dept_2': 2542.4892, 'well': 'Pukemai-1B'},
            {'dept_1': 4048.5060, 'dept_2': 5020.9704, 'well': 'Radnor-1B'},
            {'dept_1': 1042.5684, 'dept_2': 2270.9124, 'well': 'Rahi-1'},
            {'dept_1': 900.0744, 'dept_2': 3042.9708, 'well': 'Rahotu-1'},
            {'dept_1': 578.0532, 'dept_2': 2094.8904, 'well': 'Ratanui-1'},
            {'dept_1': 452.6280, 'dept_2': 1425.8544, 'well': 'Ratapiko-1'},
            {'dept_1': 85.6488, 'dept_2': 957.9864, 'well': 'Republic New Plymouth-1'},
            {'dept_1': 89.0016, 'dept_2': 907.9992, 'well': 'Republic New Plymouth-2'},
            {'dept_1': 78.6384, 'dept_2': 945.4896, 'well': 'Republic New Plymouth-4'},
            {'dept_1': 426.1104, 'dept_2': 2111.3496, 'well': 'Richmond-1'},
            {'dept_1': 595.5792, 'dept_2': 1712.9760, 'well': 'Salisbury-1 ST1'},
            {'dept_1': 950.5188, 'dept_2': 1576.4256, 'well': 'Sidewinder-1'},
            {'dept_1': 404.0124, 'dept_2': 1795.8816, 'well': 'Standish-1'},
            {'dept_1': 2745.6384, 'dept_2': 4472.4828, 'well': 'Stratford-1A'},
            {'dept_1': 569.0616, 'dept_2': 1931.2128, 'well': 'Supplejack South-1'},
            {'dept_1': 567.5376, 'dept_2': 2061.9720, 'well': 'Supplejack South-1A'},
            {'dept_1': 568.1472, 'dept_2': 2542.4892, 'well': 'Supplejack-1'},
            {'dept_1': 581.1012, 'dept_2': 1843.4304, 'well': 'Surrey-2ST1'},
            {'dept_1': 410.5656, 'dept_2': 1002.9444, 'well': 'Surville-1'},
            {'dept_1': 413.0040, 'dept_2': 1574.9016, 'well': 'Tabla-1'},
            {'dept_1': 601.5228, 'dept_2': 1560.4236, 'well': 'Takahe-1'},
            {'dept_1': 1684.6296, 'dept_2': 3702.8628, 'well': 'Takapou-1'},
            {'dept_1': 1805.6352, 'dept_2': 2497.3788, 'well': 'Tariki-4'},
            {'dept_1': 700.5828, 'dept_2': 2261.9208, 'well': 'Tariki-D1'},
            {'dept_1': 659.1300, 'dept_2': 2378.9640, 'well': 'Te Kiri-2'},
            {'dept_1': 1403.6040, 'dept_2': 3270.3516, 'well': 'Te Kumi-1'},
            {'dept_1': 755.5992, 'dept_2': 2875.7880, 'well': 'Te Popo-1'},
            {'dept_1': 2541.1176, 'dept_2': 3202.9908, 'well': 'Tipoka-1A'},
            {'dept_1': 1198.6260, 'dept_2': 2750.9724, 'well': 'Tipoka-2'},
            {'dept_1': 929.6400, 'dept_2': 1380.4392, 'well': 'Toetoe-1'},
            {'dept_1': 1951.6344, 'dept_2': 2429.8656, 'well': 'Toetoe-4B'},
            {'dept_1': 907.0848, 'dept_2': 3465.8808, 'well': 'Trapper-A1'},
            {'dept_1': 2195.6268, 'dept_2': 2481.9864, 'well': 'Tuhua-5A'},
            {'dept_1': 1959.5592, 'dept_2': 2926.3848, 'well': 'Tuhua-6'},
            {'dept_1': 604.5708, 'dept_2': 2670.3528, 'well': 'Tuhua-7'},
            {'dept_1': 1662.5316, 'dept_2': 2440.3812, 'well': 'Tuhua-8A'},
            {'dept_1': 1572.1584, 'dept_2': 3726.4848, 'well': 'Tui SW-2'},
            {'dept_1': 3126.0288, 'dept_2': 3879.4944, 'well': 'Tui-1'},
            {'dept_1': 1333.0428, 'dept_2': 3732.8856, 'well': 'Tuihu-1'},
            {'dept_1': 3355.5432, 'dept_2': 5182.9716, 'well': 'Turangi-3'},
            {'dept_1': 3588.5628, 'dept_2': 4447.9464, 'well': 'Waihapa-1A'},
            {'dept_1': 801.6240, 'dept_2': 2783.4336, 'well': 'Waihapa-H1'},
            {'dept_1': 684.5808, 'dept_2': 2876.8548, 'well': 'Warea-1'},
            {'dept_1': 2997.0984, 'dept_2': 3581.5524, 'well': 'Wharehuia-1'},
            {'dept_1': 460.0956, 'dept_2': 1607.3628, 'well': 'Windsor-1'},
            {'dept_1': 450.6468, 'dept_2': 1442.9232, 'well': 'Windsor-2'},
            {'dept_1': 450.6468, 'dept_2': 1523.3904, 'well': 'Windsor-3A'},
            {'dept_1': 1961.0832, 'dept_2': 4213.8600, 'well': 'Witiora-1'}
]

In [ ]:
for params in nan_list:
    df_test = values_nan(df_test, **params)

In [ ]:
wells = []
for well in nan_list:
    wells.append(well['well'])

In [ ]:
df_new = df_test.loc[wells]

In [ ]:
def interpolate_nan_values(group):
    for column in group.columns:
        series = group[column]

        # Find sequences of NaN values
        nan_sequences = []
        current_sequence = []
        for i in range(len(series)):
            if pd.isnull(series.iloc[i]):
                current_sequence.append(i)
            elif current_sequence:
                nan_sequences.append(current_sequence)
                current_sequence = []
        if current_sequence:
            nan_sequences.append(current_sequence)

        # Interpolate sequences of NaN values if they do not exceed 100
        for sequence in nan_sequences:
            if len(sequence) <= 100:
                # Interpolate the sequence with the upper and lower numerical values
                upper_value = series.iloc[sequence[0]-1] if sequence[0] > 0 else series.iloc[sequence[0]]
                lower_value = series.iloc[sequence[-1]+1] if sequence[-1] < len(series) - 1 else series.iloc[sequence[-1]]
                series.iloc[sequence] = np.interp(sequence, [sequence[0], sequence[-1]], [upper_value, lower_value])

        group[column] = series

    return group

df_depth_filtered = df_new.groupby(level=0).apply(interpolate_nan_values).droplevel(0)

In [ ]:
def correction_outlier(df, depth1, depth2, well, curve, value, inequality='>'):
    depth_mask = (df.index.get_level_values(1) >= depth1) & (df.index.get_level_values(1) <= depth2)
    mask = (df.index.get_level_values(0) == well) & depth_mask
    df.loc[mask, curve] = df.loc[mask, curve].fillna(0)
    if inequality == '>':
        df.loc[mask, curve] = df.loc[mask, curve].mask(df.loc[mask, curve] > value)
    elif inequality == '<':
        df.loc[mask, curve] = df.loc[mask, curve].mask(df.loc[mask, curve] < value)
    df.loc[mask, curve] = df.loc[mask, curve].interpolate(method='linear')
    df.loc[mask, curve] = df.loc[mask, curve].replace(0, np.nan)

In [ ]:
corrections_outlier_list = [{'depth1': 3000, 'depth2': 3500, 'well': 'Wharehuia-1', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 3600, 'depth2': 4400, 'well': 'Waihapa-1A', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 2000, 'depth2': 2920, 'well': 'Tuhua-6', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 2400, 'depth2': 2450, 'well': 'Tuhua-5A', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 660, 'depth2': 1000, 'well': 'Te Kiri-2', 'curve': 'NEUT', 'value': 0.96, 'inequality': '>'},
                            {'depth1': 4800, 'depth2': 5000, 'well': 'Radnor-1B', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 3400, 'depth2': 3600, 'well': 'Pukeko-1', 'curve': 'NEUT', 'value': 0.96, 'inequality': '>'},
                            {'depth1': 2600, 'depth2': 2720, 'well': 'North Tasman-1', 'curve': 'NEUT', 'value': 0, 'inequality': '<'},
                            {'depth1': 4200, 'depth2': 4470, 'well': 'Stratford-1A', 'curve': 'NEUT', 'value': 0.9, 'inequality': '>'}
]

In [ ]:
for params in corrections_outlier_list:
    correction_outlier(df_depth_filtered, **params)

In [ ]:
neut_proc = ['Toetoe-4B']
mask = df_depth_filtered.index.get_level_values(0).isin(neut_proc)
df_depth_filtered.loc[mask, 'NEUT'] = df_depth_filtered.loc[mask, 'NEUT'] / 100

In [ ]:
tests = {'GR': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_positive],
         'DENS': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_positive],
         'NEUT': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_between(0,1)],
         'DRHO': [wq.no_flat,
                  wq.no_gaps],
         'CALI': [wq.no_flat,
                  wq.no_gaps,
                  wq.all_positive],
         'SP': [wq.no_flat,
                  wq.no_gaps],
         }

In [ ]:
for well in df_depth_filtered.index.get_level_values(0).unique():
  w = df_depth_filtered.loc[well]
  wl = welly.Well({})
  for log in tests.keys():  # Проходим по логам, для которых есть тесты
        if log in w.columns:  # Проверяем, имеются ли данные по этому логe
            wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)

 # show test results in HTML table
  print('\n', '\n')
  print('WELL NAME: ' + well)
  print('CURVE QUALITY SUMMARY')
  html = wl.qc_table_html(tests)
  display(HTML(html, metadata = {'title': well,}))


 

WELL NAME: Ahuroa South B-1
CURVE QUALITY SUMMARY


<ipython-input-101-d06fd4e801b2>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)


Curve,Passed,Score,all_between,no_gaps,all_positive,no_flat
GR,3 / 3,1.000,,True,True,True
DENS,3 / 3,1.000,,True,True,True
NEUT,3 / 3,1.000,True,True,,True
DRHO,2 / 2,1.000,,True,,True
CALI,3 / 3,1.000,,True,True,True
SP,2 / 2,1.000,,True,,True



 

WELL NAME: Ahuroa-1
CURVE QUALITY SUMMARY


<ipython-input-101-d06fd4e801b2>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)


...

<ipython-input-101-d06fd4e801b2>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)


Curve,Passed,Score,all_between,no_gaps,all_positive,no_flat
GR,3 / 3,1.000,,True,True,True
DENS,3 / 3,1.000,,True,True,True
NEUT,3 / 3,1.000,True,True,,True
DRHO,2 / 2,1.000,,True,,True
CALI,3 / 3,1.000,,True,True,True
SP,2 / 2,1.000,,True,,True


<ipython-input-101-d06fd4e801b2>:8: DeprecationWarning: In a future release, the basis argument will be removed; use index instead.
  wl.data[log] = welly.Curve(w[log].values, basis=w.index.get_level_values('DEPT').values, mnemonic=log)



 

WELL NAME: Witiora-1
CURVE QUALITY SUMMARY


Curve,Passed,Score,all_between,no_gaps,all_positive,no_flat
GR,3 / 3,1.000,,True,True,True
DENS,3 / 3,1.000,,True,True,True
NEUT,3 / 3,1.000,True,True,,True
DRHO,2 / 2,1.000,,True,,True
CALI,3 / 3,1.000,,True,True,True
SP,2 / 2,1.000,,True,,True


In [ ]:
df_plot = df_depth_filtered.copy()
df_plot.reset_index(inplace=True)

In [ ]:
def create_plot(wellname, dataframe,
                curves_to_plot, depth_curve,
                log_curves=[]):

    # Count the number of tracks we need
    num_tracks = len(curves_to_plot)

    # Setup the figure and axes
    fig, ax = plt.subplots(nrows=1, ncols=num_tracks, figsize=(num_tracks*2, 10))

    # Create a super title for the entire plot
    fig.suptitle(wellname, fontsize=20, y=1.05)

    # Loop through each curve in curves_to_plot and create a track with that data
    for i, curve in enumerate(curves_to_plot):


        ax[i].plot(dataframe[curve], depth_curve)

        # Setup a few plot cosmetics
        ax[i].set_title(curve, fontsize=14, fontweight='bold')
        ax[i].grid(which='major', color='lightgrey', linestyle='-')

        # We want to pass in the deepest depth first, so we are displaying the data
        # from shallow to deep
        ax[i].set_ylim(depth_curve.max(), depth_curve.min())

        # Only set the y-label for the first track. Hide it for the rest
        if i == 0:
            ax[i].set_ylabel('DEPTH (m)', fontsize=18, fontweight='bold')
        else:
            plt.setp(ax[i].get_yticklabels(), visible = False)

        # Check to see if we have any logarithmic scaled curves
        if curve in log_curves:
            ax[i].set_xscale('log')
            ax[i].grid(which='minor', color='lightgrey', linestyle='-')

    plt.tight_layout()
    plt.show()

In [ ]:
grouped = df_plot.groupby('WELLNAME')
dfs_wells = []
wellnames = []

for well, data in grouped:
  dfs_wells.append(data)
  wellnames.append(well)
curves_to_plot = ['CALI', 'DENS', 'DRHO', 'GR', 'NEUT', 'SP']

In [ ]:
for well in range(len(wellnames)):
  create_plot(wellnames[well], dfs_wells[well],
              curves_to_plot, dfs_wells[well]['DEPT'])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df_depth_filtered.to_csv('taranaki_logs.csv', index = True)